In [5]:
"""
Code for databricks, to demo the model. 

(CPU Version tho for the broke bois)

"""

import tensorflow as tf
import pickle
import os 
import cv2
import numpy as np
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score
print(tf.__version__)

IMAGE_SIZE = (224, 224, 3)


2.7.0


In [6]:
def make_model_2():
    def build_convnet(shape=None):
        momentum = 0.9
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Conv2D(64, (3,3), input_shape=shape[1:], padding='same', activation='linear'))
        model.add(tf.keras.layers.BatchNormalization(momentum=momentum))
        model.add(tf.keras.layers.Activation('relu'))

        model.add(tf.keras.layers.MaxPool2D())

        model.add(tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='linear'))
        model.add(tf.keras.layers.BatchNormalization(momentum=momentum))
        model.add(tf.keras.layers.Activation('relu'))

        model.add(tf.keras.layers.MaxPool2D())

        model.add(tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='linear'))
        model.add(tf.keras.layers.BatchNormalization(momentum=momentum))
        model.add(tf.keras.layers.Activation('relu'))

        # flatten
        model.add(tf.keras.layers.GlobalMaxPool2D())
        return model
    shape = (90, IMAGE_SIZE[0], IMAGE_SIZE[1], IMAGE_SIZE[2])
    print('Train data shape: ', shape)

    convnet = build_convnet(shape)

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.TimeDistributed(convnet, input_shape=shape))
    model.add(tf.keras.layers.LSTM(64))

    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(.5))
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dropout(.5))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dropout(.5))
    model.add(tf.keras.layers.Dense(16, activation='relu'))

    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    return model
        

In [7]:
def make_model(): 
    inp =  tf.keras.layers.Input((None, IMAGE_SIZE[0], IMAGE_SIZE[1], IMAGE_SIZE[2])) # , ragged=True
    
    mobilenet_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
        include_top=False, weights='imagenet', pooling='max', classes=2, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
    )
    
    for k,v in mobilenet_model._get_trainable_state().items():
        k.trainable = False
    
    x = tf.keras.layers.TimeDistributed(mobilenet_model)(inp)
    x = tf.keras.layers.LSTM(64, return_sequences=False)(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    out = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

    model = tf.keras.Model(inp, out)

    model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer = tf.keras.optimizers.Adam(learning_rate=0.01), metrics = ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    return model

In [8]:
with open("../file_names_folds.pkl", 'rb') as f: 
    SEEDS, FOLD_FILES = pickle.load(f)

# train the model

fold = FOLD_FILES[0]

print('FOLD::: ', fold)

train_files = [a.strip('_') for a in fold['train']]
test_files = [a.strip('_') for a in fold['test']]

X_train = []
X_test = []

y_train = []
y_test = []

for filename in train_files:
    filename_int = int(filename.split('.mp4')[0])

    if filename_int <= 115:
        curr_y = 1
        subdir_name = 'armflapping'
    else:
        curr_y = 0
        subdir_name = 'control'

    curr_x = []
    for frame in os.listdir('../behavior_data/' + subdir_name + '/' + filename):

        frame_num = int(frame.split('.')[0])
        if frame_num > 90:
            continue

        image = cv2.imread('../behavior_data/' + subdir_name + '/' + filename + '/' + frame)
        try:
            image = image.reshape((image.shape[0], image.shape[1], image.shape[2]))
        except:
            continue

        image = cv2.resize(image, (224, 224))
        curr_x.append(image)

    len_data = len(os.listdir('../behavior_data/' + subdir_name + '/' + filename))
    if len_data < 90:
        for abc in range(len_data, 90):
            curr_x.append(np.zeros((224, 224, 3)))

    curr_x = np.array(curr_x)

    X_train.append(curr_x)
    y_train.append(curr_y)

for filename in test_files:
    filename_int = int(filename.split('.mp4')[0])

    if filename_int <= 115:
        curr_y = 1
        subdir_name = 'armflapping'
    else:
        curr_y = 0
        subdir_name = 'control'

    curr_x = []
    for frame in os.listdir('../behavior_data/' + subdir_name + '/' + filename):

        frame_num = int(frame.split('.')[0])
        if frame_num > 90:
            continue

        image = cv2.imread('../behavior_data/' + subdir_name + '/' + filename + '/' + frame)
        try:
            image = image.reshape((image.shape[0], image.shape[1], image.shape[2]))
        except:
            continue

        image = cv2.resize(image, (224, 224))
        curr_x.append(image)

    len_data = len(os.listdir('../behavior_data/' + subdir_name + '/' + filename))
    if len_data < 90:
        for abc in range(len_data, 90):
            curr_x.append(np.zeros((224, 224, 3)))

    curr_x = np.array(curr_x)

    X_test.append(curr_x)
    y_test.append(curr_y)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

FOLD:::  {'train': ['_48.mp4', '_61.mp4', '_71.mp4', '_10.mp4', '_17.mp4', '_101.mp4', '_65.mp4', '_66.mp4', '_6.mp4', '_49.mp4', '_63.mp4', '_89.mp4', '_76.mp4', '_14.mp4', '_111.mp4', '_114.mp4', '_99.mp4', '_72.mp4', '_3.mp4', '_106.mp4', '_110.mp4', '_77.mp4', '_115.mp4', '_13.mp4', '_12.mp4', '_5.mp4', '_107.mp4', '_105.mp4', '_11.mp4', '_64.mp4', '_75.mp4', '_74.mp4', '_1.mp4', '_16.mp4', '_100.mp4', '_7.mp4', '_88.mp4', '_60.mp4', '_112.mp4', '_102.mp4', '_166.mp4', '_173.mp4', '_138.mp4', '_144.mp4', '_130.mp4', '_140.mp4', '_163.mp4', '_162.mp4', '_154.mp4', '_172.mp4', '_175.mp4', '_165.mp4', '_164.mp4', '_120.mp4', '_124.mp4', '_142.mp4', '_160.mp4', '_174.mp4', '_153.mp4', '_134.mp4', '_128.mp4', '_159.mp4', '_169.mp4', '_143.mp4', '_157.mp4', '_156.mp4', '_137.mp4', '_118.mp4', '_178.mp4', '_176.mp4', '_167.mp4', '_170.mp4', '_116.mp4', '_119.mp4', '_181.mp4', '_179.mp4', '_171.mp4', '_158.mp4', '_152.mp4', '_186.mp4'], 'test': ['_2.mp4', '_29.mp4', '_98.mp4', '_104.mp4', 

In [9]:


model = make_model()

model.compile(loss = tf.keras.losses.BinaryCrossentropy(), 
                optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), 
                metrics = [['accuracy', tf.keras.metrics.Precision(name="precision"), tf.keras.metrics.Recall(name="recall")]])

history = model.fit(X_train, 
                    y_train, 
                    validation_data = (X_test, y_test),
                    batch_size = 16,
                epochs = 60)


2022-01-02 11:04:43.373187: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/60
5/5 [==============================] - 130s 22s/step - loss: 0.7520 - accuracy: 0.5000 - precision: 0.5000 - recall: 0.6750 - val_loss: 0.6367 - val_accuracy: 0.7000 - val_precision: 0.6250 - val_recall: 1.0000
Epoch 2/60
5/5 [==============================] - 99s 19s/step - loss: 0.6002 - accuracy: 0.7125 - precision: 0.6977 - recall: 0.7500 - val_loss: 0.5405 - val_accuracy: 0.7000 - val_precision: 0.6250 - val_recall: 1.0000
Epoch 3/60
5/5 [==============================] - 85s 18s/step - loss: 0.5758 - accuracy: 0.7375 - precision: 0.7209 - recall: 0.7750 - val_loss: 0.5057 - val_accuracy: 0.7500 - val_precision: 0.6667 - val_recall: 1.0000
Epoch 4/60
5/5 [==============================] - 82s 17s/step - loss: 0.4992 - accuracy: 0.7625 - precision: 0.7692 - recall: 0.7500 - val_loss: 0.4974 - val_accuracy: 0.8000 - val_precision: 0.7143 - val_recall: 1.0000
Epoch 5/60
5/5 [==============================] - 89s 19s/step - loss: 0.5080 - accuracy: 0.7375 - precision: 0.743

In [11]:
model.save("MBNet")

INFO:tensorflow:Assets written to: MBNet/assets


INFO:tensorflow:Assets written to: MBNet/assets
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


# save the model